# データパイプライン実行ノートブック (v2: クラスベース)

このノートブックは、Object Storageからドキュメントを取得し、
テキスト抽出、チャンク分割、Embedding、DB保存を行います。

**前提条件**:
- `.env`ファイルが設定済み（DB接続、OCI認証情報）
- Object Storageバケットにドキュメントがアップロード済み
- データベーステーブル（source_documents, chunks）が作成済み

## 1. 必要なライブラリのインポート

In [ ]:
####################################
###  プロジェクトルートをパスに追加  ###
####################################

import sys
import os

# プロジェクトルートをPythonパスに追加
# notebooksディレクトリから実行されている場合、親ディレクトリをパスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"✓ Project root added to Python path: {project_root}")


####################################
########  必要なライブラリ群 ##########
####################################

# Oracle Database
import oracledb

# 設定読み込み
from src.config import ConfigLoader

# データパイプラインクラス
from src.data_pipeline.document_loader import DocumentLoader
from src.data_pipeline.text_extractor import TextExtractor
from src.data_pipeline.text_chunker import TextChunker
from src.data_pipeline.embedding_generator import EmbeddingGenerator
from src.data_pipeline.document_writer import DocumentWriter
from src.data_pipeline.data_pipeline import DataPipeline

print("✓ All imports successful")

## 2. 設定の読み込みとDB接続

In [ ]:
################
#  設定と接続  #
################

# ConfigLoaderを使用して設定を読み込み
config = ConfigLoader()
config.load_env()

# Oracle Database接続
db_params = config.get_db_params()
connection = oracledb.connect(**db_params)

# OCI設定
oci_config = config.get_oci_config()

# Object Storage設定
os_config = config.get_object_storage_config()
bucket_name = os_config['bucket_name']

# Object Storageクライアントを作成してnamespaceを取得
os_client = config.get_object_storage_client()
namespace = os_client.get_namespace().data

print(f"✓ OCIリソースの情報を読み込みました")
print(f"  - Database: {connection}")
print(f"  - Bucket: {bucket_name}")
print(f"  - Namespace: {namespace}")

## 3. チューニングパラメータ（必要に応じて編集）

In [ ]:
############################
#  チューニングパラメータ  #
############################

# デフォルト値を取得
app_config = config.get_app_config()

# チャンクサイズ（文字数）
# デフォルトから変更する場合は、数値に変更 (例: chunk_size = 1000)
chunk_size = app_config['chunk_size']  # デフォルト: 500

# チャンクのオーバーラップ（文字数）
# デフォルトから変更する場合は、数値に変更 (例: chunk_overlap = 100)
chunk_overlap = app_config['chunk_overlap']  # デフォルト: 50

print(f"✓ チューニングパラメータ設定値")
print(f"  - Chunk Size: {chunk_size}")
print(f"  - Chunk Overlap: {chunk_overlap}")

## 4. パイプライン構築（コンポーネント初期化とオーケストレータ作成）

In [ ]:
############################
#  パイプラインコンポーネント  # 
############################

# Phase 1: DocumentLoader（Object Storageからのダウンロード）
loader = DocumentLoader(
    oci_config=oci_config,
    bucket_name=bucket_name,
    namespace=namespace
)

# Phase 2: TextExtractor（PDF/TXT/CSVからのテキスト抽出）
extractor = TextExtractor()

# Phase 3: TextChunker（テキストの分割）
chunker = TextChunker(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# Phase 4: EmbeddingGenerator（ベクトル埋め込み生成）
embedding_gen = EmbeddingGenerator()

# Phase 5: DocumentWriter（DB保存）
writer = DocumentWriter(connection)

print("✓ All 5 components initialized")
print(f"  1. DocumentLoader (bucket: {bucket_name}, namespace: {namespace})")
print(f"  2. TextExtractor (supports: pdf, txt, csv)")
print(f"  3. TextChunker (size: {chunk_size}, overlap: {chunk_overlap})")
print(f"  4. EmbeddingGenerator (model: auto-configured)")
print(f"  5. DocumentWriter (connection: active)")


############################
#  プログレスコールバック  # 
############################

# オプション: プログレス追跡用のコールバック関数
def progress_callback(filename: str, status: str):
    """
    ファイル処理の進捗を表示
    
    Args:
        filename: ファイル名
        status: 'success', 'failed', 'skipped' のいずれか
    """
    status_emoji = {
        'success': '✓',
        'failed': '✗',
        'skipped': '⊘'
    }
    emoji = status_emoji.get(status, '•')
    print(f"  {emoji} {filename}: {status}")


############################
#  パイプラインオーケストレータ  # 
############################

# Phase 6: DataPipeline（全コンポーネントの統合）
pipeline = DataPipeline(
    loader=loader,
    extractor=extractor,
    chunker=chunker,
    embedding_gen=embedding_gen,
    writer=writer,
    progress_callback=progress_callback  # プログレス表示を有効化
)

print("✓ DataPipeline orchestrator created")
print("  - Error isolation enabled (individual file failures won't stop processing)")
print("  - Progress tracking enabled")

## 5. Object Storageからファイル一覧を取得

In [ ]:
############################
#  ファイル一覧の取得  # 
############################

# Object Storageからすべてのファイルパスを取得
file_paths = loader.list_files()

print(f"✓ Found {len(file_paths)} files in bucket '{bucket_name}'")
print("\nFile list:")
for i, path in enumerate(file_paths, 1):
    print(f"  {i}. {path}")

## 6. すべてのファイルを一括処理

In [ ]:
############
# メイン処理 #
############

print("\n" + "="*60)
print("Starting Pipeline Processing...")
print("="*60 + "\n")

# すべてのファイルをパイプライン処理
# process_all()は各ファイルを順次処理し、エラーが発生しても継続
result = pipeline.process_all(file_paths)

# 処理結果のサマリーを表示
print("\n" + "="*60)
print("Processing Complete")
print("="*60)
print(f"\n📊 Summary:")
print(f"  ✓ Success: {result.successful} files")
print(f"  ✗ Failed:  {result.failed} files")
print(f"  ⊘ Skipped: {result.skipped} files")
print(f"  📁 Total:   {result.total_files} files")
print(f"  📦 Chunks:  {result.total_chunks} chunks")
print(f"  ⏱️  Time:    {result.elapsed_time:.2f} seconds")

# 各ファイルの処理結果を表示
if result.processed_docs:
    print(f"\n📋 Detailed Results:")
    for doc in result.processed_docs:
        status_emoji = {'success': '✓', 'failed': '✗', 'skipped': '⊘'}.get(doc.status, '•')
        print(f"\n  {status_emoji} {doc.filename} ({doc.status})")
        
        if doc.status == 'success':
            doc_id_hex = doc.document_id.hex() if doc.document_id else 'N/A'
            print(f"      Document ID: {doc_id_hex}")
            print(f"      Chunks: {doc.chunks_saved}")
        elif doc.error:
            print(f"      Error: {doc.error}")

## 7. DB接続のクローズ

In [ ]:
################
#  後処理  # 
################

# データベース接続を明示的にクローズ
connection.close()
print("✓ Database connection closed")